In [ ]:
from keras.layers import Lambda
import tensorflow as tf
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, UpSampling2D, AveragePooling2D, Dropout
from keras.models import Model
from glob import glob
import re
import os.path
import scipy.misc
import numpy as np
import random
import matplotlib.pyplot as plt
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import keras.backend as K

import imageio
import skimage.transform


def SubpixelConv2D(input_shape, scale=4):
    def subpixel_shape(input_shape):
        dims = [input_shape[0],
                input_shape[1] * scale,
                input_shape[2] * scale,
                int(input_shape[3] / (scale ** 2))]
        output_shape = tuple(dims)
        return output_shape

    def subpixel(x):
        return tf.nn.depth_to_space(x, scale)

    return Lambda(subpixel, output_shape=subpixel_shape, name='subpixel')
    

image_shape = (320,480)
data_dir = './directory'       # Change to work directory
batch_size = 15                # Change according to GPU power process / Testesd with NVIDIA GPU GTX 1080


###############################################################################################################

def generator(data_folder, image_shape, batch_size):
    image_paths = glob(os.path.join(data_folder, 'images', '*.png'))
    label_paths = {
        re.sub(r'_road_', '_', os.path.basename(path)): path
        for path in glob(os.path.join(data_folder, 'labels', '*_road_*.png'))}
    
    background_color = np.array([255, 0, 255])

    random.shuffle(image_paths)
    while 1: 
        for batch_i in range(0, len(image_paths), batch_size):
            images = []
            gt_images = []
            for image_file in image_paths[batch_i:batch_i+batch_size]:
                gt_image_file =re.sub('images','labels',image_file)
                image = scipy.misc.imread(image_file)
                gt_image = scipy.misc.imread(gt_image_file)
                gt_bg = np.all(gt_image == background_color, axis=2) 
                gt_bg = gt_bg.reshape(*gt_bg.shape, 1)
                gt_image = np.concatenate((gt_bg, np.invert(gt_bg)), axis=2)
                images.append(image)
                gt_images.append(gt_image)
                
            X = np.array(images)
            Y = np.array(gt_images)
            yield (X, Y)
        
train_generator = generator(os.path.join(data_dir, 'training'), image_shape, batch_size)
validation_generator = generator(os.path.join(data_dir, 'validation'), image_shape, batch_size)
test_generator = generator(os.path.join(data_dir, 'test'), image_shape, batch_size)

def f1 (y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    tp = K.sum(y_true_f * y_pred_f)
    fp = K.sum(K.clip(K.clip(y_pred_f+y_true_f,0,1)-y_true_f,0,1))
    fn = K.sum(K.clip(K.clip(y_pred_f+y_true_f,0,1)-y_pred_f,0,1))
    Pr = tp/(tp+fp)
    Re = tp/(tp+fn)
    f1t = 2*(Pr*Re)/(Pr+Re+K.epsilon())
    return f1t

In [ ]:
namefile = 'CrackIPN'  # Name of trained model

size = (5,5)

def CrackIPN(img_rows, img_cols, img_channels):
    
    x = Input(shape=(img_rows, img_cols, img_channels))
    
    conv1 = Conv2D(64, size, padding='same', activation='relu')(x)
    conv2 = Conv2D(64, size, padding='same', activation='relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(128, size, padding='same', activation='relu')(pool1)
    conv4 = Conv2D(128, size, padding='same', activation='relu')(conv3)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    conv5 = Conv2D(256, size, padding='same', activation='relu')(pool2)
    conv6 = Conv2D(256, size, padding='same', activation='relu')(conv5)
    
    sub1 = SubpixelConv2D(conv6.shape,scale = 4)(conv6)
    
    conv7 = Conv2D(64, size, padding='same', activation='relu')(sub1)
    conv8 = Conv2D(64, size, padding='same', activation='relu')(conv7)
    
    merge2 = concatenate([conv8, conv2], axis=3)
    
    conv9 = Conv2D(128, size, padding='same', activation='relu')(merge2)
    
    y = Conv2D(2, (1, 1), activation='sigmoid')(conv9)

    return Model(inputs=x, outputs=y)

In [ ]:
model = CrackIPN(image_shape[0],image_shape[1],3)
model.summary()
model.load_weights(namefile + '.h5')

In [ ]:
import time
import matplotlib.pyplot as plt
from PIL import Image


th_segment = 0.1   # Select beta
def gen_img (nombre,cantidad):
    data_folder = './CrackIPN/test/images/' 
    k = cantidad
    filelist = glob(os.path.join(data_folder, '*.png'))
    i = 1
    inicio = time.time()

    for image_file in filelist[0:k]:
        img_original = imageio.imread(image_file)
        img = scipy.misc.imresize(img_original, image_shape) 
        y = model.predict(np.expand_dims(img, axis=0), batch_size=1)  
        mask = np.stack([y[0,:,:,0] > th_segment,np.zeros(image_shape),np.zeros(image_shape)], axis=2)
        masked = np.ma.masked_array(mask, img).astype('float32')

        z = str(i).zfill(3)
        scipy.misc.imsave('./CrackIPN/test/results/'+z+'_'+nombre+'.png',masked) # masked

        i += 1

    fin = time.time()

# -----------------------------------------------------------------
def obt_f (nombre,cantidad):
    inicio = time.time()
    data_folder = './CrackIPN/test/results/'
    k = cantidad
    filelist = glob(os.path.join(data_folder, '*.png'))
    i = 1
    f1t = 0
    prt = 0
    ret = 0
    a = []
    for image_file in filelist[0:k]:
        img = imageio.imread(image_file)
        img = np.asarray(img)
        if (nombre == 'crackipn'):
            a = imageio.imread('./CrackIPN/test/labels/'+str(i+350).zfill(3)+'.png')

        # Ground truth
        b = np.empty([len(a),len(a[0])])
        for m in range (len(a)):
            for n in range (len (a[0])):
                if a[m][n][0] == 0:
                    b[m][n] = 0
                else:
                    b[m][n] = 1

        # Model result
        d = np.empty([len(img),len(img[0])])  # c
        for m in range (len(img)):
            for n in range (len (img[0])):
                if img[m][n][0] == 0:
                    d[m][n] = 0
                else:
                    d[m][n] = 1
                    
        # Postprocessing -------------------------------
        
        er = 2
        cc = np.empty([len(img),len(img[0])]) 
        for p in range(len(d)):
            for q in range(len(d[p])):
                if d[p-er:p+er,q-er:q+er].sum() <= 5: # 5
                    cc[p][q] = 0
                else:
                    cc[p][q] = 1
        
        c = np.empty([len(img),len(img[0])]) 
        
        c = cc
        
        fin = time.time()
                 
        scipy.misc.imsave('./CrackIPN/test/postprocess/'+str(i)+'_'+nombre+'.png',c)
        #--------------------------------------------------------------------

        tp = 0
        fp = 0
        fn = 0
        h = 2   
        for r in range(len(c)):
            for o in range(len(c[0])):
                if c[r][o] == 1 and b[r-h:r+h,o-h:o+h].sum() >= 1:
                    tp += 1
                elif c[r][o] == 1 and b[r][o] == 0:
                    fp += 1
                elif c[r][o] == 0 and b[r][o] == 1:
                    fn += 1
        pr = tp/(tp+fp +K.epsilon())
        re = tp/(tp+fn +K.epsilon())
        f1 = 2*(pr*re)/(pr+re +K.epsilon())

        f1t = (f1t+f1)
        prt = prt+pr
        ret = ret+re

        i += 1
    
    print ('Dataset '+ nombre + ' -- PrT: ', prt/cantidad, ' ReT: ', ret/cantidad, 'F1T: ', f1t/cantidad)
    return f1t/cantidad

    
    
####################################################################################################################
for i in range (3,4):
    th_segment = i/10.0
    gen_img (nombre='crackipn',cantidad=50)
    f_uri = obt_f(nombre = 'crackipn', cantidad = 50)
    print ('th_segment: ', th_segment)

print ('End')